In [11]:
import torch
import torch.nn as nn
import dgl
import dgl.nn as dglnn
# # Test on converting from lstm output to graph input
# batch_size = 64
# hidden_dim = 8
# num_agents = 60
# lstm_output = torch.randn((batch_size * num_agents, hidden_dim)).requires_grad_()

batch_size = 64
seq_len = 5
num_agents = 60
features = 2
hidden_dim = 8

input_tensor = torch.randn((batch_size, seq_len, num_agents, features))
permuted = input_tensor.permute(0, 2, 1, 3).contiguous() # (batch_size, num_agents, seq_len, features)
lstm_input = permuted.view(batch_size * num_agents, seq_len, features)

lstm_layer = nn.LSTM(input_size=features, 
                hidden_size=hidden_dim, 
                num_layers=1, 
                batch_first=True)

out, (hn, cn) = lstm_layer(lstm_input)
print(hn.shape)

reshaped = hn.view(batch_size, num_agents, hidden_dim)
print(reshaped.shape)

graph_list = []

test_list = range(num_agents)
edges = list(combinations(test_list, 2))
start_nodes = [i[0] for i in edges]
end_nodes = [i[1] for i in edges]

for i in range(batch_size):
    x = reshaped[i]
    s = torch.tensor(start_nodes)
    e = torch.tensor(end_nodes)
    g = dgl.graph((s, e))
    g.ndata['x'] = reshaped[i]

    graph_list.append(g)

batched_graphs = dgl.batch(graph_list)

print(batched_graphs.batch_num_nodes())


torch.Size([1, 3840, 8])
torch.Size([64, 60, 8])


In [10]:
from itertools import combinations
import torch
test_list = range(64)

edges = list(combinations(test_list, 2))
start_nodes = [i[0] for i in edges]
end_nodes = [i[1] for i in edges]

print(len(start_nodes))
print(len(end_nodes))

# b = torch.tensor(a)

# print(len(a))
# print(b)


2016
2016


In [7]:
from dgl.data import MiniGCDataset
dataset = MiniGCDataset(80, 10, 20)
graph, label = dataset[0]

# print(dataset[0])
# print(dataset[0][0].ndata['h'].shape)
g = dataset[0][0]
h = g.in_degrees().view(-1, 1).float()
print(g)
print(h)

Graph(num_nodes=15, num_edges=45,
      ndata_schemes={}
      edata_schemes={})
tensor([[3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.],
        [3.]])
